In [1]:
import matplotlib.pyplot as plt
import numpy as np
import random
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import tensorflow_datasets as tfds

In [2]:
import os
os.environ['TF_FORCE_GPU_ALLOW_GROWTH'] = 'true'

In [3]:
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
  try:
    # Currently, memory growth needs to be the same across GPUs
    for gpu in gpus:
      tf.config.experimental.set_memory_growth(gpu, True)
    logical_gpus = tf.config.experimental.list_logical_devices('GPU')
    print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
  except RuntimeError as e:
    # Memory growth must be set before GPUs have been initialized
    print(e)


1 Physical GPUs, 1 Logical GPUs


In [4]:
learning_rate = 0.003
meta_step_size = 0.25

inner_batch_size = 8
eval_batch_size = 8

meta_iters = 10 # 2000
eval_iters = 5 # repetitions
inner_iters = 4 # repetitions

eval_interval = 1 #meta_iter % eval_interval == 0이면 eval. 1이면 매번 한다는 뜻.
train_shots = 64
eval_shots = 16 # eval
classes = 2

In [5]:
import cv2
class Dataset:
    # This class will facilitate the creation of a few-shot dataset
    # from the Omniglot dataset that can be sampled from quickly while also
    # allowing to create new labels at the same time.
    def __init__(self, mode):
        # Download the tfrecord files containing the omniglot data and convert to a
        # dataset.
        self.mode = mode
        self.labels = range(11)

                
    def read_img(self, real_label, shots):
        ds = []
        path = os.path.join(os.getcwd(), 'image8', self.mode, str(real_label))
        files = os.listdir(path)
        sp = random.choices(files, k=shots)
        for s in sp:
            image = cv2.imread(path + "/" + s, cv2.IMREAD_COLOR)  # uint8 image
            norm_image = cv2.normalize(image, None, alpha=0, beta=1, norm_type=cv2.NORM_MINMAX, dtype=cv2.CV_32F)
            ds.append(norm_image)
        return ds
    
    def get_mini_dataset(
        self, batch_size, repetitions, train_shots, eval_shots, num_classes, split=False
    ):
        temp_labels = np.zeros(shape=(num_classes * train_shots))
        temp_images = np.zeros(shape=(num_classes * train_shots, 299, 299, 3))
        if split:
            test_labels = np.zeros(shape=(num_classes * eval_shots))
            test_images = np.zeros(shape=(num_classes * eval_shots, 299, 299, 3))

        # Get a random subset of labels from the entire label set.
        label_subset = random.sample(self.labels, k=num_classes)
        for class_idx, class_obj in enumerate(label_subset):
            # Use enumerated index value as a temporary label for mini-batch in
            # few shot learning.
            temp_labels[class_idx * train_shots : (class_idx + 1) * train_shots] = class_idx
            # If creating a split dataset for testing, select an extra sample from each
            # label to create the test dataset.
            if split:
                test_labels[class_idx * eval_shots : (class_idx + 1) * eval_shots] = class_idx
                test_images[
                    class_idx * eval_shots : (class_idx + 1) * eval_shots
                ] = self.read_img(label_subset[class_idx], eval_shots)
                temp_images[
                    class_idx * train_shots : (class_idx + 1) * train_shots
                ] = self.read_img(label_subset[class_idx], train_shots)
            else:
                # For each index in the randomly selected label_subset, sample the
                # necessary number of images.
                temp_images[
                    class_idx * train_shots : (class_idx + 1) * train_shots
                ] = self.read_img(label_subset[class_idx], train_shots)

        dataset = tf.data.Dataset.from_tensor_slices(
            (temp_images.astype(np.float32), temp_labels.astype(np.int32))
        )
        dataset = dataset.shuffle(100).batch(batch_size).repeat(repetitions)
        if split:
            return dataset, test_images.astype(np.float32), test_labels.astype(np.int32)
        return dataset


import urllib3

urllib3.disable_warnings()  # Disable SSL warnings that may happen during download.
train_dataset = Dataset(mode='train')
test_dataset = Dataset(mode='val')

In [6]:
img_rows, img_cols, img_channel = 299, 299, 3

base_model = tf.keras.applications.Xception(weights='imagenet', include_top=True, input_shape=(img_rows, img_cols, img_channel))

In [7]:
add_model = tf.keras.Sequential()
add_model.add(tf.keras.layers.Flatten(input_shape=base_model.output_shape[1:]))
#add_model.add(tf.keras.layers.Dropout(rate = 0.8))
#add_model.add(tf.keras.layers.Dropout(rate = 0.8))

#add_model.add(tf.keras.layers.Dropout(rate = 0.8))
#add_model.add(tf.keras.layers.Dense(units=256, activation=tf.nn.relu))
add_model.add(tf.keras.layers.Dense(units=2, activation=tf.nn.softmax))

model = tf.keras.Model(inputs=base_model.input, outputs=add_model(base_model.output))
model.compile(loss='categorical_crossentropy', optimizer=tf.keras.optimizers.Adam(lr=0.0001),
              metrics=['accuracy'])
optimizer = tf.keras.optimizers.Adam(lr=0.0001)
#model.summary()

In [8]:
training = []
testing = []
for meta_iter in range(meta_iters):
    frac_done = meta_iter / meta_iters
    cur_meta_step_size = (1 - frac_done) * meta_step_size
    # Temporarily save the weights from the model.
    old_vars = model.get_weights()
    # Get a sample from the full dataset.
    mini_dataset = train_dataset.get_mini_dataset(
        inner_batch_size, inner_iters, train_shots, eval_shots, classes
    )
    for images, labels in mini_dataset:
        with tf.GradientTape() as tape:
            preds = model(images)
            loss = keras.losses.sparse_categorical_crossentropy(labels, preds)
        grads = tape.gradient(loss, model.trainable_weights)
        optimizer.apply_gradients(zip(grads, model.trainable_weights))
    new_vars = model.get_weights()
    # Perform SGD for the meta step.
    for var in range(len(new_vars)):
        new_vars[var] = old_vars[var] + (
            (new_vars[var] - old_vars[var]) * cur_meta_step_size
        )
    # After the meta-learning step, reload the newly-trained weights into the model.
    model.set_weights(new_vars)
    # Evaluation loop
    if meta_iter % eval_interval == 0:
        accuracies = []
        for dataset in (train_dataset, test_dataset):
            # Sample a mini dataset from the full dataset.
            train_set, test_images, test_labels = dataset.get_mini_dataset(
                eval_batch_size, eval_iters, train_shots, eval_shots, classes, split=True
            )
            old_vars = model.get_weights()
            # Train on the samples and get the resulting accuracies.
            for images, labels in train_set:
                with tf.GradientTape() as tape:
                    preds = model(images)
                    loss = keras.losses.sparse_categorical_crossentropy(labels, preds)
                grads = tape.gradient(loss, model.trainable_weights)
                optimizer.apply_gradients(zip(grads, model.trainable_weights))
            test_preds = model.predict(test_images)
            test_preds = tf.argmax(test_preds, axis=1).numpy()
            num_correct = (test_preds == test_labels).sum()
            # Reset the weights after getting the evaluation accuracies.
            model.set_weights(old_vars)
            accuracies.append(num_correct / classes)
        training.append(accuracies[0])
        testing.append(accuracies[1])
        #if meta_iter % 100 == 0:
        print(
            "batch %d: train=%f test=%f" % (meta_iter, accuracies[0], accuracies[1])
        )

ResourceExhaustedError: OOM when allocating tensor with shape[8,728,19,19] and type float on /job:localhost/replica:0/task:0/device:GPU:0 by allocator GPU_0_bfc [Op:Conv2D]

In [ ]:
# First, some preprocessing to smooth the training and testing arrays for display.
window_length = 100
train_s = np.r_[
    training[window_length - 1 : 0 : -1], training, training[-1:-window_length:-1]
]
test_s = np.r_[
    testing[window_length - 1 : 0 : -1], testing, testing[-1:-window_length:-1]
]
w = np.hamming(window_length)
train_y = np.convolve(w / w.sum(), train_s, mode="valid")
test_y = np.convolve(w / w.sum(), test_s, mode="valid")

# Display the training accuracies.
x = np.arange(0, len(test_y), 1)
plt.plot(x, test_y, x, train_y)
plt.legend(["test", "train"])
plt.grid()
plt.show()

In [ ]:
train_set, test_images, test_labels = dataset.get_mini_dataset(
    eval_batch_size, eval_iters, train_shots, eval_shots, classes, split=True
)
for images, labels in train_set:
    with tf.GradientTape() as tape:
        preds = model(images)
        loss = keras.losses.sparse_categorical_crossentropy(labels, preds)
    grads = tape.gradient(loss, model.trainable_weights)
    optimizer.apply_gradients(zip(grads, model.trainable_weights))
test_preds = model.predict(test_images)
test_preds = tf.argmax(test_preds).numpy()
num_correct = (test_preds == test_labels).sum()
print(num_correct / classes)

In [ ]:
add_model = tf.keras.Sequential()
add_model.add(tf.keras.layers.Dense(units=11, activation=tf.nn.softmax))

e_model = tf.keras.Model(inputs=model.input, outputs=add_model(model.output))
e_model.compile(loss='categorical_crossentropy', optimizer=tf.keras.optimizers.Adam(lr=0.0001),
              metrics=['accuracy'])

In [ ]:
batch_size = 8
epochs = 10

train_datagen = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1/255.)
val_datagen = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1/255.)
test_datagen = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1/255.)

train_set = train_datagen.flow_from_directory('image8/train',
                                                 target_size = (299, 299),
                                                 batch_size = batch_size,
                                                 class_mode = 'categorical')

val_set = val_datagen.flow_from_directory('image8/val',
                                                 target_size = (299, 299),
                                                 batch_size = batch_size,
                                                 class_mode = 'categorical')


callback = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=3)

history = e_model.fit_generator(
    train_set,
    steps_per_epoch= 4742 // batch_size,
    epochs=epochs,
    validation_data=val_set,
    validation_steps = 527 // batch_size,
    callbacks=[callback]
)

In [ ]:
import matplotlib.pyplot as plt

plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])

plt.title('model auc')
plt.ylabel('acc')
plt.xlabel('epoch')
plt.legend(['train','test'], loc='upper left')
plt.show()

plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])

plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train','test'], loc='upper left')
plt.show()

In [ ]:
test_set = test_datagen.flow_from_directory('image8/test',
                                                 target_size = (299, 299),
                                                 batch_size = 8,
                                                 class_mode = 'categorical')
score = model.evaluate(test_set, steps = 28501 // 8)